In [55]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import re
from sklearn import svm
from sklearn.metrics import precision_recall_fscore_support


## Loading Data

__dataset is split into train and test, with 11000 tweets in training set and the rest in test dataset__

In [3]:

df=pd.read_csv("final_dataset.csv")
test_data = df[11000:].copy()
data = df[:11000].copy()


## Data Preprocessing

In [4]:
data.isnull().values.any()

False

In [5]:
def tweet_processing(raw_tweet):
    letters_only=re.sub("[^a-zA-Z]"," ",raw_tweet)
    words=letters_only.lower().split()
    stops=set(stopwords.words("english"))
    m_w=[w for w in words if not w in stops]
    return (" ".join(m_w))

In [6]:
num_tweets=data["Tweets"].size
clean_tweet=[]
for i in range(0,num_tweets):
    clean_tweet.append(tweet_processing(data["Tweets"][i]))
data["Tweets"]=clean_tweet 


num_tweets_test=test_data["Tweets"].size
clean_tweet_test=[]
for i in range(num_tweets,num_tweets+num_tweets_test):
    clean_tweet_test.append(tweet_processing(test_data["Tweets"][i]))
test_data["Tweets"]=clean_tweet_test

## Baseline model : SVM

In [9]:
X_train, X_test_svm, Y_train, Y_test_svm = train_test_split(df.Tweets, df.Class, test_size=0.2)

In [12]:
vectorizer=CountVectorizer(analyzer = "word",tokenizer = None,preprocessor = None,stop_words = None,max_features = 5000)

train_data_features=vectorizer.fit_transform(X_train)
train_data_features=train_data_features.toarray()

test_data_features=vectorizer.transform(X_test_svm)
test_data_features=test_data_features.toarray()

In [13]:
#SVM with linear kernel
clf=svm.SVC(kernel='linear',C=1.0)
print ("Training")
clf.fit(train_data_features,Y_train)

print ("Testing")
predicted=clf.predict(test_data_features)
accuracy=np.mean(predicted==Y_test_svm)
print ("Accuracy: ",accuracy)

Training
Testing
Accuracy:  0.8618101545253863


__precision,recall,F1 score for svm model__

In [106]:
from sklearn.metrics import precision_recall_fscore_support
score_svm=precision_recall_fscore_support(Y_test, predicted, average='weighted')
print(score_svm)

(0.6150747931093572, 0.6313465783664459, 0.622672398462044, None)


C:\Users\Srishti\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Deep Learning Models

__creating one hot vector for classes(labels).__ 

In [33]:
data['sexism']=0
data['racism']=0
data['none']=0

data['sexism'] = np.where(data['Class'] == 'sexism', 1, 0)
data['racism'] = np.where(data['Class'] == 'racism', 1, 0)
data['none'] = np.where(data['Class'] == 'none', 1, 0)
#data.head()
columns=['sexism','racism','none']
y=data[columns].values
#print(y.shape)


test_data['sexism']=0
test_data['racism']=0
test_data['none']=0

test_data['sexism'] = np.where(test_data['Class'] == 'sexism', 1, 0)
test_data['racism'] = np.where(test_data['Class'] == 'racism', 1, 0)
test_data['none'] = np.where(test_data['Class'] == 'none', 1, 0)
#data.head()
columns=['sexism','racism','none']
y_test=test_data[columns].values

__Tokenizing words__

In [18]:
from nltk.tokenize import word_tokenize
#text1 = "It's true that the chicken was the best bamboozler in the known multiverse."
#tokens = word_tokenize(data['Tweets'])
data['tokenized_sents'] = data.apply(lambda column: word_tokenize(column['Tweets']), axis=1)
test_data['tokenized_sents'] = test_data.apply(lambda column: word_tokenize(column['Tweets']), axis=1)
df['tokenized_sents'] = df.apply(lambda column: word_tokenize(column['Tweets']), axis=1)


__Creating word embeddings__

In [23]:
from gensim.models import Word2Vec
# define training data
sentences = df['tokenized_sents']
# train model
model = Word2Vec(sentences,size=200,window =4,min_count=1,sg=1)
#print(model)
words = list(model.wv.vocab)
#print(len(words))
model.save('model.bin')

#model = Word2Vec.load('model.bin')
#print(model)


__creating embedding matrix__

In [24]:
embedding_matrix = np.zeros((vocab_size,200))

for i in range(0,len(words)):
    embedding_vector = model[words[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
embedding_matrix[vocab_size-1]= np.random.normal(scale=0.6, size=(200, ))
    
#print(embedding_matrix)

C:\Users\Srishti\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


__Using Keras to train LSTM model__

In [56]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import LeakyReLU
from keras.models import load_model

__resizing each tweet to size 50__ 

In [21]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['tokenized_sents'])
sequences = tokenizer.texts_to_sequences(data['tokenized_sents'])
X_t = pad_sequences(sequences, maxlen=50)
vocab_size = 22194
#print(vocab_size)
tokenizer.fit_on_texts(test_data['tokenized_sents'])
test_sequences = tokenizer.texts_to_sequences(test_data['tokenized_sents'])
X_test = pad_sequences(test_sequences, maxlen=50)


__calculating a user's tendency towards racism, sexism, neutrality by taking the ratio of number of tweets marked as a particular label and total number of tweets of that user. This is done for each label(sexism, racism and neutral)__

In [25]:
count1=[]
print(data['User Id'].values)
for i in data['User Id'].unique():
    #print(i)
    count1.append((data['User Id']== i).sum())
#print(count1)
neutral_count=[]
sexism_count=[]
racism_count=[]
for i in data['User Id'].unique():
    neutral_count.append(len(data[(data['User Id']==i) & (data['Class']=='none')]))
    sexism_count.append(len(data[(data['User Id']==i) & (data['Class']=='sexism')]))
    racism_count.append(len(data[(data['User Id']==i) & (data['Class']=='racism')])) 
    
count_test=[]
#print(data['User Id'].values)
for i in test_data['User Id'].unique():
    #print(i)
    count_test.append((test_data['User Id']== i).sum())
#print(count1)
neutral_count_test=[]
sexism_count_test=[]
racism_count_test=[]
for i in test_data['User Id'].unique():
    #neutral_count.append((data['User Id']==i) & (data['Class']== 'none').sum())
    neutral_count_test.append(len(test_data[(test_data['User Id']==i) & (test_data['Class']=='none')]))
    sexism_count_test.append(len(test_data[(test_data['User Id']==i) & (test_data['Class']=='sexism')]))
    racism_count_test.append(len(test_data[(test_data['User Id']==i) & (test_data['Class']=='racism')])) 
    


[949380854 297877558 272704749 ... 289846547  29424561 289846547]


In [26]:
X_len = len(data['User Id'].unique())

In [27]:
ratio_sexism=[]
ratio_neutral=[]
ratio_racism=[]

for i in range(0,X_len):
    ratio_sexism.append(sexism_count[i]/count1[i])
    ratio_racism.append(racism_count[i]/count1[i])
    ratio_neutral.append(neutral_count[i]/count1[i])
        
#make a column for ratio of eachc class

ratio_sexism_test=[]
ratio_neutral_test=[]
ratio_racism_test=[]

for i in range(0,len(test_data['User Id'].unique())):
    ratio_sexism_test.append(sexism_count_test[i]/count_test[i])
    ratio_racism_test.append(racism_count_test[i]/count_test[i])
    ratio_neutral_test.append(neutral_count_test[i]/count_test[i])


In [28]:
j=0
for i in data['User Id'].unique():
    
    #print(i)
    data.loc[data['User Id'] == i,'tendency_sexual'] = ratio_sexism[j]
    data.loc[data['User Id'] == i,'tendency_racism'] = ratio_racism[j]
    data.loc[data['User Id'] == i,'tendency_neutral'] = ratio_neutral[j]
    j=j+1

k=0
for i in test_data['User Id'].unique():
    
    #print(i)
    test_data.loc[test_data['User Id'] == i,'tendency_sexual'] = ratio_sexism_test[k]
    test_data.loc[test_data['User Id'] == i,'tendency_racism'] = ratio_racism_test[k]
    test_data.loc[test_data['User Id'] == i,'tendency_neutral'] = ratio_neutral_test[k]
    k=k+1

In [30]:
tendency_sexism=data['tendency_sexual'].as_matrix()
tendency_racism=data['tendency_racism'].as_matrix()
tendency_neutral=data['tendency_neutral'].as_matrix()
tendency_sexism=tendency_sexism.reshape(len(tendency_sexism),1)
tendency_racism=tendency_sexism.reshape(len(tendency_racism),1)
tendency_neutral=tendency_sexism.reshape(len(tendency_neutral),1)
print(tendency_sexism.shape)

tendency_sexism_test=test_data['tendency_sexual'].as_matrix()
tendency_racism_test=test_data['tendency_racism'].as_matrix()
tendency_neutral_test=test_data['tendency_neutral'].as_matrix()
tendency_sexism_test=tendency_sexism_test.reshape(len(tendency_sexism_test),1)
tendency_racism_test=tendency_sexism_test.reshape(len(tendency_racism_test),1)
tendency_neutral_test=tendency_sexism_test.reshape(len(tendency_neutral_test),1)
#print(tendency_sexism.shape)

(11000, 1)


C:\Users\Srishti\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\Users\Srishti\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\Srishti\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Srishti\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':
C:\Users\Srishti\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.pa

__concatenating word vectors with tendencies of users calculated above for each label__

In [64]:
#print(X_t)
X1_t=np.concatenate((X_t, tendency_sexism), axis=1)
X1_t=np.concatenate((X1_t, tendency_racism), axis=1)
X1_t=np.concatenate((X1_t, tendency_neutral), axis=1)
print(X1_t.shape)


X1_test=np.concatenate((X_test, tendency_sexism_test), axis=1)
X1_test=np.concatenate((X1_test, tendency_racism_test), axis=1)
X1_test=np.concatenate((X1_test, tendency_neutral_test), axis=1)


(11000, 53)


__Training LSTM Model__

In [92]:
inp = Input(shape=(53,))
x = Embedding(vocab_size,200,weights=[embedding_matrix])(inp)
#print(x.values)
x = (LSTM(50, return_sequences=True, dropout=0.2, recurrent_dropout=0.4))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50)(x)
x=LeakyReLU(alpha=0.02)(x)
x = Dropout(0.2)(x)
x = Dense(3)(x)
x=LeakyReLU(alpha=0.02)(x)
model1 = Model(inputs=inp, outputs=x)
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.fit(X1_t,y, batch_size=32, epochs=5, validation_split=0.1);

Train on 9900 samples, validate on 1100 samples
Epoch 1/5
9900/9900 [==============================] - 58s 6ms/step - loss: 0.4043 - acc: 0.8264 - val_loss: 0.2793 - val_acc: 0.8912
Epoch 2/5
9900/9900 [==============================] - 52s 5ms/step - loss: 0.2755 - acc: 0.9053 - val_loss: 0.4068 - val_acc: 0.9142
Epoch 3/5
9900/9900 [==============================] - 51s 5ms/step - loss: 0.2153 - acc: 0.9056 - val_loss: 0.3078 - val_acc: 0.9115
Epoch 4/5
9900/9900 [==============================] - 52s 5ms/step - loss: 0.1474 - acc: 0.8856 - val_loss: 0.5323 - val_acc: 0.8652
Epoch 5/5
9900/9900 [==============================] - 52s 5ms/step - loss: 0.1172 - acc: 0.8606 - val_loss: 0.5688 - val_acc: 0.8091


In [65]:
#saving model
model1.save('model_lstm.h5')  

__training a bideirectional LSTM model__

In [93]:
inp_bi = Input(shape=(53,))
x_bi = Embedding(vocab_size,200,weights=[embedding_matrix])(inp_bi)
#print(x.values)
x_bi = Bidirectional(LSTM(50, return_sequences=True, dropout=0.2, recurrent_dropout=0.4))(x_bi)
x_bi = GlobalMaxPool1D()(x_bi)
x_bi = Dense(50)(x_bi)
x_bi=LeakyReLU(alpha=0.02)(x_bi)
x_bi = Dropout(0.2)(x_bi)
x_bi = Dense(3)(x_bi)
x_bi=LeakyReLU(alpha=0.02)(x_bi)
model_bi = Model(inputs=inp_bi, outputs=x_bi)
model_bi.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_bi.fit(X1_t,y, batch_size=32, epochs=5, validation_split=0.1);
model_bi.save('model_bi.h5') 

Train on 9900 samples, validate on 1100 samples
Epoch 1/5
9900/9900 [==============================] - 72s 7ms/step - loss: 0.4401 - acc: 0.8079 - val_loss: 0.2553 - val_acc: 0.9033
Epoch 2/5
9900/9900 [==============================] - 64s 6ms/step - loss: 0.3585 - acc: 0.8989 - val_loss: 0.3089 - val_acc: 0.9248
Epoch 3/5
9900/9900 [==============================] - 64s 7ms/step - loss: 0.2007 - acc: 0.9061 - val_loss: 0.2847 - val_acc: 0.9173
Epoch 4/5
9900/9900 [==============================] - 67s 7ms/step - loss: 0.4475 - acc: 0.8161 - val_loss: 0.6964 - val_acc: 0.8045
Epoch 5/5
9900/9900 [==============================] - 66s 7ms/step - loss: 0.1546 - acc: 0.8701 - val_loss: 0.4702 - val_acc: 0.8542


__Calculating  precision,recall,F1score, of LSTM and bidirectional model__

In [94]:
predicted_lstm = model1.predict(X1_test)
predicted_bi = model_bi.predict(X1_test)

In [95]:
dataframe_lstm = pd.DataFrame.from_records(predicted_lstm)
dataframe_bi=pd.DataFrame.from_records(predicted_bi)

__predicted class is stored in a column of dataframe__

In [111]:

print(dataframe_lstm['predClass'])


0        none
1        none
2        none
3        none
4        none
5      Sexism
6        none
7        none
8        none
9        none
10       none
11       none
12       none
13       none
14       none
15       none
16       none
17       none
18       none
19       none
20       none
21       none
22       none
23       none
24       none
25       none
26       none
27       none
28       none
29       none
        ...  
295      none
296    Sexism
297    Sexism
298      none
299      none
300      none
301      none
302      none
303      none
304      none
305      none
306      none
307      none
308      none
309      none
310      none
311      none
312    Sexism
313    Sexism
314      none
315    Sexism
316      none
317    Sexism
318      none
319      none
320      none
321      none
322      none
323      none
324      none
Name: predClass, Length: 325, dtype: object


__bidirectional lstm results__

In [112]:
print(dataframe_bi['predClass'])

0        none
1        none
2        none
3        none
4        none
5        none
6        none
7        none
8        none
9        none
10       none
11       none
12       none
13       none
14       none
15       none
16       none
17       none
18       none
19       none
20       none
21       none
22       none
23       none
24       none
25       none
26       none
27       none
28       none
29       none
        ...  
295      none
296      none
297      none
298      none
299      none
300      none
301      none
302      none
303      none
304      none
305      none
306      none
307      none
308      none
309      none
310      none
311      none
312    Sexism
313    Sexism
314      none
315    Sexism
316      none
317      none
318      none
319      none
320      none
321      none
322      none
323      none
324      none
Name: predClass, Length: 325, dtype: object


__get label function creates appropriate labels according to  predictions__

In [96]:

def get_label(df):
    if ((df[0] >df[1]) & (df[0] > df[2])):
        return 'Sexism'
    elif ((df[1] >df[0]) & (df[1] > df[2])):
        return 'Racism'
    else:
        return 'none'


dataframe_lstm['predClass'] = dataframe_lstm.apply(lambda row: get_label(row), axis=1)
dataframe_bi['predClass'] = dataframe_bi.apply(lambda row: get_label(row), axis=1)



__LSTM score__

In [103]:
score_lstm=precision_recall_fscore_support(test_data['Class'],dataframe_lstm['predClass'],average='weighted')
print(score_lstm)

(0.8760505166475316, 0.7723076923076924, 0.8209144701452393, None)


C:\Users\Srishti\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Srishti\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


__Bidirectional LSTM score__

In [105]:
score_bi=precision_recall_fscore_support(test_data['Class'],dataframe_bi['predClass'],average='weighted')
print(score_bi)

(0.8706521160500399, 0.8276923076923077, 0.8486288753405109, None)


C:\Users\Srishti\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Srishti\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


__precision recall and f1 scores for svm and deep learning models__:


In [108]:
summary = [[score_svm[0],score_svm[1],score_svm[2]], [score_lstm[0],score_lstm[1],score_lstm[2] ],[score_bi[0],score_bi[1],score_bi[2]]]
score=pd.DataFrame(summary, columns=["Precision", "Recall","F1-score"])
score.rename(index={0:'SVM',1:'LSTM',2:'Bidirectional LSTM'}, inplace=True)
print(score)

                    Precision    Recall  F1-score
SVM                  0.615075  0.631347  0.622672
LSTM                 0.876051  0.772308  0.820914
Bidirectional LSTM   0.870652  0.827692  0.848629
